In [1]:
import pandas as pd
import numpy as np

file_path = "C:/Users/patri/OneDrive/Documents/!HSG/DSF/project/" 

In [3]:
# read the primary data set from 2020, rename columns
packages_agg = pd.read_csv(file_path + "packages_both_years.csv")
packages_agg.rename(columns={"MONAT": "month", "PRODUKT_GRUPPE": "product", "PLZ": "postcode", "WOCHENTAG": "weekday", "ANZAHL": "packages"}, inplace=True)


,month,weekday,postcode,product,packages
0,01/01/2020,DI,9000,ECO,4251
1,01/01/2020,DI,9000,SEM,274
2,01/01/2020,DI,9000,PRI,9350
20,01/01/2020,DI,9001,PRI,40
21,01/01/2020,DI,9001,ECO,50
...,...,...,...,...,...
86528,01/12/2021,DI,9657,ECO,93
86529,01/12/2021,DI,9657,SEM,4
86544,01/12/2021,DI,9658,ECO,152
86545,01/12/2021,DI,9658,SEM,8


In [4]:
# we have 3 variants of dispatching (ECO, PRI, SEM), we want to get the aggregate number of packages
# thus we we group by postcode, month, weekday and then take the sum of the dispatchments
packages_grouped = packages_agg.groupby(["postcode", "month", "weekday"]).agg(sum=("packages", "sum"))

# we reset the index and rename the sum column to packages
packages_grouped.reset_index(inplace=True)
packages_grouped.rename(columns={"sum": "packages"}, inplace=True)


In [5]:
# we need the bfs number, which is different from the postal code, 
# to merge with our other features! Thus we first merge our data with the bfs number data.

# read the data with the bfs numbers and rename the columns
bfs_number = pd.read_csv(file_path + "features_csv/plz_bfs.csv", sep=";") 
bfs_number.rename(columns={"PLZ": "postcode", "BFS-Nr": "bfs_no"}, inplace = True)
bfs_number = bfs_number.loc[(bfs_number["Kantonskürzel"] == "SG")]

# ESSENTIAL step; we need to remove the duplicate entries in the postcode column
# since otherwise we'll have an indistict mapping from postcode to bfs number and thus
# additional rows will be added that originally did not exist (see more below).
bfs_number.drop_duplicates(subset=["postcode"], inplace=True)

# merge the data: we use a left merge since the important data is the "packages" data set
# and we only want to add those bfs numbers to which we have a corresponding postcode.
mergeA = pd.merge(packages_grouped, bfs_number[["postcode", "bfs_no"]], how = "left", on="postcode")

# # In our data we have municipalities from St. Gallen, Appenzell IR/AR, Liechtenstein, and 
# some of Thurgau. However, we did not find any data for the municipalities of Appenzell IR/AR,
# Liechtenstein, and Thurgau for our features, even when we called them :/
# So we drop the rows of those municipalities for which we do not have data, i.e., a bfs number!
mergeA.dropna(subset="bfs_no", inplace = True)

# we aggregate the packages by the bfs number
master_data = mergeA.groupby(["bfs_no", "month", "weekday"]).agg(sum=("packages", "sum"))

# reset the index and rename the column
master_data.reset_index(inplace=True)
master_data.rename(columns={"sum": "packages"}, inplace = True)

# convert the bfs number and the packages back to integers
master_data[["packages", "bfs_no"]] = master_data[["packages", "bfs_no"]].astype(int)

# save the master data
# master_data.to_csv(file_path + "master_data.csv")

In [5]:
# here we inspected our mistake, we had multiple rows containing the same postcode 
# and the same bfs number...fallacy!!
# test = bfs_number.groupby("postcode").count().reset_index().sort_values("bfs_no")
# test.loc[(test["bfs_no"] > 1) & (test["postcode"] >= 9000)]

In [6]:
# Jonathan (Chassot) showing us how the mapping error is possible when we have multiple rows
# that contain the same postcode, however it was a fallacy to think we have multiple bfs 
# numbers per postcode, the reason was we had multiple rows containing the same postcode 
# AND bfs number!

# df1 = pd.DataFrame({"postcode": [9000,8000,6000], "packages": [1,2,3]})
# df2 = pd.DataFrame({"postcode": [9000, 9000], "bfs-nr": [1,2]})
# pd.merge(df1, df2, how="outer")

In [12]:
# divide the master dataset into 2020 and 2021
packages_2021 = master_data.loc[(master_data["month"] == "01/01/2021") | (master_data["month"] == "01/02/2021") | (master_data["month"] == "01/03/2021") | (master_data["month"] == "01/04/2021") | (master_data["month"] == "01/05/2021") | (master_data["month"] == "01/06/2021") | (master_data["month"] == "01/07/2021") | (master_data["month"] == "01/08/2021") | (master_data["month"] == "01/09/2021") | (master_data["month"] == "01/10/2021") | (master_data["month"] == "01/11/2021") | (master_data["month"] == "01/12/2021")].reset_index(drop=True)
packages_2020 = master_data.loc[(master_data["month"] == "01/01/2020") | (master_data["month"] == "01/02/2020") | (master_data["month"] == "01/03/2020") | (master_data["month"] == "01/04/2020") | (master_data["month"] == "01/05/2020") | (master_data["month"] == "01/06/2020") | (master_data["month"] == "01/07/2020") | (master_data["month"] == "01/08/2020") | (master_data["month"] == "01/09/2020") | (master_data["month"] == "01/10/2020") | (master_data["month"] == "01/11/2020") | (master_data["month"] == "01/12/2020")].reset_index(drop=True)

# since we're not interested whether it's the particular december in 2021, but just whether it's december or not!
packages_2021["month"] = packages_2021["month"].map({"01/01/2021": "January", "01/02/2021": "February", "01/03/2021": "March", "01/04/2021": "April", "01/05/2021": "May", "01/06/2021": "June", "01/07/2021": "July", "01/08/2021": "August", "01/09/2021": "September", "01/10/2021": "October", "01/11/2021": "November", "01/12/2021": "December"})
packages_2020["month"] = packages_2020["month"].map({"01/01/2020": "January", "01/02/2020": "February", "01/03/2020": "March", "01/04/2020": "April", "01/05/2020": "May", "01/06/2020": "June", "01/07/2020": "July", "01/08/2020": "August", "01/09/2020": "September", "01/10/2020": "October", "01/11/2020": "November", "01/12/2020": "December"})


In [13]:
# read the data for the mean age, keep only relevant columns and rename them 
age = pd.read_csv(file_path + "features_csv/mean_age.csv", usecols=["BFS_NR", "2021", "2020"])
age.rename(columns={"BFS_NR": "bfs_no", "2021": "mean_age_2021", "2020": "mean_age_2020"}, inplace=True)
# merge the master data with the age data
merge_age_2020 = pd.merge(packages_2020, age[["bfs_no", "mean_age_2020"]], how="left", on="bfs_no")
merge_age_2021 = pd.merge(packages_2021, age[["bfs_no", "mean_age_2021"]], how="left", on="bfs_no")


In [14]:
# read the data for the permanent total population and for the % of population in certain
# age range and rename columns
from glob import glob

# use the function glob to catch all population data files and easily combine them into one population dataset (with concat)
popdata_list = sorted(glob(file_path + "features_csv/population_data/population*"))

# merge all the data we have for the years 2020 to 2021 into one merged population df
merged_pop = pd.concat((pd.read_csv(file, usecols=["BFS_NR", "2021", "2020"]) for file in popdata_list), ignore_index=True, axis=1)
# we drop the addtional columnns that contain the bfs number as well
merged_pop.drop(columns=[3, 6, 9, 12, 15, 18], inplace=True)
# rename the columns
merged_pop.rename(columns={0: "bfs_no", 1: "total_pop2021", 2: "total_pop2020", 4: "share2021_0to14", 5: "share2020_0to14", 7: "share2021_15to19", 8: "share2020_15to19", 10: "share2021_20to39", 11:"share2020_20to39", 13: "share2021_40to64", 14: "share2020_40to64", 16: "share2021_65to79", 17: "share2020_65to79", 19: "share2021_above80", 20: "share2020_above80"}, inplace = True)

# merge the data on the primary datasets
merge_pop_2020 = pd.merge(merge_age_2020, merged_pop[["bfs_no", "total_pop2020", "share2020_0to14", "share2020_15to19", "share2020_20to39", "share2020_40to64", "share2020_65to79", "share2020_above80"]], how="left", on="bfs_no") 
merge_pop_2021 = pd.merge(merge_age_2021, merged_pop[["bfs_no", "total_pop2021", "share2021_0to14", "share2021_15to19", "share2021_20to39", "share2021_40to64", "share2021_65to79", "share2021_above80"]], how="left", on="bfs_no")



In [15]:
# read and rename columns of foreigner share
foreigners = pd.read_csv("C:/Users/patri/Downloads/features_csv/foreigners_share.csv", usecols=["BFS_NR", "2021", "2020"])
foreigners.rename(columns={"BFS_NR": "bfs_no", "2021": "foreigners_2021", "2020": "foreigners_2020"}, inplace=True)

# merge
merge_for_2020 = pd.merge(merge_pop_2020, foreigners[["bfs_no", "foreigners_2020"]], how="left", on="bfs_no")
merge_for_2021 = pd.merge(merge_pop_2021, foreigners[["bfs_no", "foreigners_2021"]], how="left", on="bfs_no")

In [16]:
# read and rename columns of share of catholics, protestants and other religions
religion_data = sorted(glob(file_path + "features_csv/religion_data/religion*"))

merged_religion = pd.concat((pd.read_csv(file, usecols=["BFS_NR", "2021", "2020"]) for file in religion_data), ignore_index=True, axis=1)
merged_religion.rename(columns={0: "bfs_no", 1:"catholics2021", 2: "catholics2020", 4:"other_relig_2021", 5: "other_relig_2020", 7: "protestants_2021", 8: "protestants_2020"}, inplace= True)
merged_religion.drop(columns=[3, 6], inplace = True)

# merge, merge, merge
merge_relig2020 = pd.merge(merge_for_2020, merged_religion[["bfs_no", "catholics2020", "other_relig_2020", "protestants_2020"]], how="left", on="bfs_no")
merge_relig2021 = pd.merge(merge_for_2021, merged_religion[["bfs_no", "catholics2021", "other_relig_2021", "protestants_2021"]], how="left", on="bfs_no")


In [17]:
# read and rename columns of the unemployment data
unemployment = pd.read_csv(file_path + "features_csv/unemployment.csv", usecols=["BFS_NR", "2021", "2020"])
unemployment.rename(columns={"BFS_NR": "bfs_no", "2021": "unemployment2021", "2020": "unemployment2020"}, inplace=True)

# merge
merge_unemp2020 = pd.merge(merge_relig2020, unemployment[["bfs_no", "unemployment2020"]], how="left", on="bfs_no")
merge_unemp2021 = pd.merge(merge_relig2021, unemployment[["bfs_no", "unemployment2021"]], how="left", on="bfs_no")


In [27]:
# read and rename columns of the tax power data
tax_power = pd.read_csv(file_path + "features_csv/tax_power.csv", usecols=["BFS_NR", "2021", "2020"])
tax_power.rename(columns={"BFS_NR": "bfs_no", "2021": "tax_power2021", "2020": "tax_power2020"}, inplace = True)

# merge
merge_tax2020 = pd.merge(merge_unemp2020, tax_power[["bfs_no", "tax_power2020"]], how="left", on="bfs_no")
merge_tax2021 = pd.merge(merge_unemp2021, tax_power[["bfs_no", "tax_power2021"]], how="left", on="bfs_no")
#merge_tax2021.rename(columns={'tax_power2021': 'taxpower'}, inplace = True)
merge_tax2021


,bfs_no,month,weekday,packages,mean_age,total_pop,share_0to14,share_15to19,share_20to39,share_40to64,share_65to79,share_above80,share_foreigners,share_catholics,share_other_relig,share_protestants,unemployment_rate,tax_power2021
0,3201,January,DI,490,39.5,1'376,20.9,4.8,25.2,33.9,10.8,4.5,9.7,55.3,24.3,20.3,0.8,91.1
1,3201,January,DO,464,39.5,1'376,20.9,4.8,25.2,33.9,10.8,4.5,9.7,55.3,24.3,20.3,0.8,91.1
2,3201,January,FR,355,39.5,1'376,20.9,4.8,25.2,33.9,10.8,4.5,9.7,55.3,24.3,20.3,0.8,91.1
3,3201,January,MI,493,39.5,1'376,20.9,4.8,25.2,33.9,10.8,4.5,9.7,55.3,24.3,20.3,0.8,91.1
4,3201,January,MO,267,39.5,1'376,20.9,4.8,25.2,33.9,10.8,4.5,9.7,55.3,24.3,20.3,0.8,91.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4258,3444,December,DO,1874,40.2,3'569,19.9,6.2,22.0,35.8,11.2,4.9,8.2,57.4,24.,18.6,0.9,86.7
4259,3444,December,FR,1682,40.2,3'569,19.9,6.2,22.0,35.8,11.2,4.9,8.2,57.4,24.,18.6,0.9,86.7
4260,3444,December,MI,1969,40.2,3'569,19.9,6.2,22.0,35.8,11.2,4.9,8.2,57.4,24.,18.6,0.9,86.7
4261,3444,December,MO,338,40.2,3'569,19.9,6.2,22.0,35.8,11.2,4.9,8.2,57.4,24.,18.6,0.9,86.7


In [19]:
# read and rename columns of tertiary education 
tertiary_edu = pd.read_csv("C:/Users/patri/Downloads/features_csv/share_tertiary_edu.csv", usecols=["BFS_NR", "2020"])
tertiary_edu.rename(columns={"BFS_NR": "bfs_no", "2020": "share_tertiary"}, inplace=True)

# merge
merge_edu2020 = pd.merge(merge_tax2020, tertiary_edu, how="left", on="bfs_no")

In [20]:
# read and rename columns of the number of firms data
firms = pd.read_csv(file_path + "features_csv/firms.csv", usecols=["BFS_NR", "2020"])
firms.rename(columns={"BFS_NR": "bfs_no", "2020": "firms"}, inplace=True)

# merge and change type to intger
merge_firms2020 = pd.merge(merge_edu2020, firms, how="left", on="bfs_no")
merge_firms2020["firms"] = merge_firms2020["firms"].str.replace("'", "",).astype(int)



In [28]:
# we had to rename the columns with the corresponding year in oder to merge it with the
# right data half. Now, we need to rename them again in order to have them concatenated
# on the same columns!

# merge_firms2020.columns
merge_firms2020.rename(columns={'mean_age_2020': 'mean_age',
       'total_pop2020': 'total_pop', 'share2020_0to14': 'share_0to14', 'share2020_15to19': 'share_15to19',
       'share2020_20to39':  'share_20to39', 'share2020_40to64': 'share_40to64', 'share2020_65to79': 'share_65to79',
       'share2020_above80': 'share_above80', 'foreigners_2020': 'share_foreigners', 'catholics2020': 'share_catholics',
       'other_relig_2020': 'share_other_relig', 'protestants_2020': 'share_protestants', 'unemployment2020': 'unemployment_rate',
       'tax_power2020': 'taxpower'}, inplace = True)


# merge_unemp2021.columns
merge_tax2021.rename(columns={'mean_age_2021': 'mean_age',
       'total_pop2021': 'total_pop', 'share2021_0to14': 'share_0to14', 'share2021_15to19': 'share_15to19',
       'share2021_20to39':  'share_20to39', 'share2021_40to64': 'share_40to64', 'share2021_65to79': 'share_65to79',
       'share2021_above80': 'share_above80', 'foreigners_2021': 'share_foreigners', 'catholics2021': 'share_catholics',
       'other_relig_2021': 'share_other_relig', 'protestants_2021': 'share_protestants', 'unemployment2021': 'unemployment_rate',
       'tax_power2021': 'taxpower'}, inplace = True)

# concatenate the two data sets
concat_data = pd.concat([merge_firms2020, merge_tax2021], join="outer", ignore_index=True)

concat_data

,bfs_no,month,weekday,packages,mean_age,total_pop,share_0to14,share_15to19,share_20to39,share_40to64,share_65to79,share_above80,share_foreigners,share_catholics,share_other_relig,share_protestants,unemployment_rate,taxpower,share_tertiary,firms
0,3201,January,DI,403,39.0,1'389,20.9,5.1,26.1,33.8,9.4,4.8,8.4,57.,21.7,21.2,0.8,90.9,59.9,106.0
1,3201,January,DO,359,39.0,1'389,20.9,5.1,26.1,33.8,9.4,4.8,8.4,57.,21.7,21.2,0.8,90.9,59.9,106.0
2,3201,January,FR,357,39.0,1'389,20.9,5.1,26.1,33.8,9.4,4.8,8.4,57.,21.7,21.2,0.8,90.9,59.9,106.0
3,3201,January,MI,346,39.0,1'389,20.9,5.1,26.1,33.8,9.4,4.8,8.4,57.,21.7,21.2,0.8,90.9,59.9,106.0
4,3201,January,MO,230,39.0,1'389,20.9,5.1,26.1,33.8,9.4,4.8,8.4,57.,21.7,21.2,0.8,90.9,59.9,106.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8567,3444,December,DO,1874,40.2,3'569,19.9,6.2,22.0,35.8,11.2,4.9,8.2,57.4,24.,18.6,0.9,86.7,NaN,NaN
8568,3444,December,FR,1682,40.2,3'569,19.9,6.2,22.0,35.8,11.2,4.9,8.2,57.4,24.,18.6,0.9,86.7,NaN,NaN
8569,3444,December,MI,1969,40.2,3'569,19.9,6.2,22.0,35.8,11.2,4.9,8.2,57.4,24.,18.6,0.9,86.7,NaN,NaN
8570,3444,December,MO,338,40.2,3'569,19.9,6.2,22.0,35.8,11.2,4.9,8.2,57.4,24.,18.6,0.9,86.7,NaN,NaN


In [29]:
# merge the coordinates (we need them to compute the distance to the nearest city)
merge_geo = pd.merge(concat_data, bfs_number[["bfs_no", "E", "N"]].drop_duplicates(subset="bfs_no"), how="left", on="bfs_no")

import geopy.geocoders 
from geopy.distance import geodesic as GD

r_cities = pd.read_csv(file_path + "features_csv/relevant_cities.csv")

for i in range(merge_geo.shape[0]):
    coords_1 = (merge_geo.N[i], merge_geo.E[i])
    city_list = []
    
    for k in range(r_cities.shape[0]):
        coords_2 = (r_cities.lat[k], r_cities.long[k])
        city_list.append(GD(coords_1, coords_2).km)
    
    merge_geo.at[i, "distance_to_nearest_city"] = min(city_list)

In [30]:
# checking if all the column features have the right format 
for col in merge_geo.columns:
    print(f"{col}: has type {merge_geo[col].dtype}")

# ugh.. some features have type object let's change it to float
wrong_type = ["total_pop", "share_catholics", "share_other_relig", "share_protestants"]
for i in range(len(wrong_type)):
    merge_geo[wrong_type[i]] = merge_geo[wrong_type[i]].str.replace("'", "").astype(float)

# save new master dataset with all features merged!
# merge_geo.to_csv("C:/Users/patri/Downloads/packages_final.csv", index=False)

# THE FINAL DATASET LET'S GO
packages_final = pd.read_csv(file_path + "packages_final.csv")
packages_final.drop(columns=["bfs_no", "E", "N"], inplace=True)

bfs_no: has type int32
month: has type object
weekday: has type object
packages: has type int32
mean_age: has type float64
total_pop: has type object
share_0to14: has type float64
share_15to19: has type float64
share_20to39: has type float64
share_40to64: has type float64
share_65to79: has type float64
share_above80: has type float64
share_foreigners: has type float64
share_catholics: has type object
share_other_relig: has type object
share_protestants: has type object
unemployment_rate: has type float64
taxpower: has type object
share_tertiary: has type float64
firms: has type float64
E: has type float64
N: has type float64
distance_to_nearest_city: has type float64
